### Execução do algoritmo Apriori - Biblioteca mlxtend
http://rasbt.github.io/mlxtend/

# Bibliotecas

In [475]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
from decimal import Decimal

# Carregamento dos Dados - .csv

In [476]:
anoArq = '2020'

In [477]:
# endereco = '../BaseDadosSelecionados/'
# endereco2 = 'Agrupado_Ocorrencia'

In [478]:
nomeArq = 'ES_datatran' + anoArq +'_Org_Apriori'

In [479]:
endereco= 'PorAno/'
# endereco= 'PorData/'

In [480]:
data = pd.read_csv(endereco + nomeArq + '.csv', encoding='ISO-8859-1', sep=';', header=None)

In [481]:
data.shape

(2268, 12)

In [482]:
data.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11
0,dia_semana=Quarta-feira,br=101.0,municipio=SOORETAMA,causa_acidente=Ultrapassagem Indevida,tipo_acidente=Colisão frontal,classificacao_acidente=Com Vítimas Feridas,fase_dia=Pleno dia,sentido_via=Decrescente,condicao_metereologica=Sol,tipo_pista=Simples,tracado_via=Não Informado,uso_solo=Não
1,dia_semana=Quarta-feira,br=101.0,municipio=ATILIO VIVACQUA,causa_acidente=Defeito Mecânico no Veículo,tipo_acidente=Saída de leito carroçável,classificacao_acidente=Sem Vítimas,fase_dia=Pleno dia,sentido_via=Decrescente,condicao_metereologica=Sol,tipo_pista=Simples,tracado_via=Curva,uso_solo=Não
2,dia_semana=Quarta-feira,br=447.0,municipio=VILA VELHA,causa_acidente=Falta de Atenção do Pedestre,tipo_acidente=Atropelamento de Pedestre,classificacao_acidente=Com Vítimas Feridas,fase_dia=Plena Noite,sentido_via=Crescente,condicao_metereologica=Céu Claro,tipo_pista=Dupla,tracado_via=Reta,uso_solo=Sim
3,dia_semana=Quarta-feira,br=101.0,municipio=SOORETAMA,causa_acidente=Falta de Atenção à Condução,tipo_acidente=Saída de leito carroçável,classificacao_acidente=Sem Vítimas,fase_dia=Pleno dia,sentido_via=Crescente,condicao_metereologica=Céu Claro,tipo_pista=Simples,tracado_via=Reta,uso_solo=Não
4,dia_semana=Quinta-feira,br=262.0,municipio=CARIACICA,causa_acidente=Falta de Atenção à Condução,tipo_acidente=Colisão lateral,classificacao_acidente=Com Vítimas Feridas,fase_dia=Pleno dia,sentido_via=Crescente,condicao_metereologica=Céu Claro,tipo_pista=Dupla,tracado_via=Reta,uso_solo=Sim


# Carregamento dos dados em uma matriz (Importante)

In [483]:
%%time
# data.shape[0] LINHAS e data.shape[1] COLUNAS

dataset = []
for i in range(0, data.shape[0]):
    dataset.append([str(data.values[i,j]) for j in range(0, data.shape[1])])

Wall time: 979 ms


In [484]:
# dataset

# Tratamento para a execução do algoritmo Apriori

http://rasbt.github.io/mlxtend/user_guide/preprocessing/TransactionEncoder/

In [485]:
%%time
te = TransactionEncoder()

Wall time: 0 ns


## 1º Transformação - Representação para uma matriz de boleanos

In [486]:
%%time
te_ary = te.fit(dataset).transform(dataset)
df = pd.DataFrame(te_ary, columns=te.columns_)

Wall time: 51 ms


In [487]:
# df

## 2ª Transformação - Representação esparsa
Exemplo 3 - Trabalhando com representações esparsas - http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/apriori/ 

In [488]:
# %%time
# oht_ary = te.fit(dataset).transform(dataset, sparse=True)
# sparse_df = pd.DataFrame.sparse.from_spmatrix(oht_ary, columns=te.columns_)

In [489]:
# sparse_df

# Execução do algoritmo apriori

In [490]:
numMinSupport = 0.1

In [491]:
%%time
# sparse_df
frequent_itemsets = apriori(df, min_support=numMinSupport, use_colnames=True)

Wall time: 276 ms


In [492]:
# frequent_itemsets

In [493]:
# metric = lift

In [494]:
%%time
association_rules_saida = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.8)
# association_rules_saida = association_rules(frequent_itemsets)

Wall time: 50 ms


In [495]:
saidaRecords = pd.DataFrame(association_rules_saida)

## Método para contabilizaro número de consequentes e antecedentes em cada regra

In [496]:
# Cria duas novas colunas contabilizando os antecedentes e consequents
saidaRecords["antecedent_len"] = saidaRecords["antecedents"].apply(lambda x: len(x))
saidaRecords["consequent_len"] = saidaRecords["consequents"].apply(lambda x: len(x))

## Filtro

In [497]:
saidaRecords = saidaRecords[ (saidaRecords['antecedent_len'] >= 2 ) & (saidaRecords['consequent_len'] >= 1) ] # & (saidaRecords['support'] >= 0.5) 

In [498]:
# saidaRecords = saidaRecords[(saidaRecords['support'] >= 0.6)]

In [499]:
saidaRecords.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len,consequent_len
32,(causa_acidente=Desobediência às normas de trâ...,(classificacao_acidente=Com Vítimas Feridas),0.123016,0.833774,0.105820,0.860215,1.031712,0.003253,1.189153,2,1
33,"(br=101.0, causa_acidente=Falta de Atenção à C...",(classificacao_acidente=Com Vítimas Feridas),0.292328,0.833774,0.253086,0.865762,1.038365,0.009351,1.238288,2,1
34,"(tipo_pista=Dupla, causa_acidente=Falta de Ate...",(br=101.0),0.118166,0.707672,0.101411,0.858209,1.212721,0.017788,2.061682,2,1
35,"(br=101.0, condicao_metereologica=Céu Claro)",(classificacao_acidente=Com Vítimas Feridas),0.395503,0.833774,0.324956,0.821628,0.985432,-0.004804,0.931903,2,1
36,"(br=101.0, condicao_metereologica=Nublado)",(classificacao_acidente=Com Vítimas Feridas),0.148148,0.833774,0.124339,0.839286,1.006610,0.000817,1.034294,2,1


In [500]:
saidaRecords.max()

antecedents           (uso_solo=Não, sentido_via=Crescente, tipo_pis...
consequents                (classificacao_acidente=Com Vítimas Feridas)
antecedent support                                             0.430335
consequent support                                             0.833774
support                                                        0.361552
confidence                                                            1
lift                                                            1.76961
leverage                                                      0.0717993
conviction                                                          inf
antecedent_len                                                        4
consequent_len                                                        2
dtype: object

## Salvando os resultados do algoritmo em um .csv

In [501]:
saidaRecords.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len,consequent_len
32,(causa_acidente=Desobediência às normas de trâ...,(classificacao_acidente=Com Vítimas Feridas),0.123016,0.833774,0.105820,0.860215,1.031712,0.003253,1.189153,2,1
33,"(br=101.0, causa_acidente=Falta de Atenção à C...",(classificacao_acidente=Com Vítimas Feridas),0.292328,0.833774,0.253086,0.865762,1.038365,0.009351,1.238288,2,1
34,"(tipo_pista=Dupla, causa_acidente=Falta de Ate...",(br=101.0),0.118166,0.707672,0.101411,0.858209,1.212721,0.017788,2.061682,2,1
35,"(br=101.0, condicao_metereologica=Céu Claro)",(classificacao_acidente=Com Vítimas Feridas),0.395503,0.833774,0.324956,0.821628,0.985432,-0.004804,0.931903,2,1
36,"(br=101.0, condicao_metereologica=Nublado)",(classificacao_acidente=Com Vítimas Feridas),0.148148,0.833774,0.124339,0.839286,1.006610,0.000817,1.034294,2,1


## Substituição de '.' para ','

In [502]:
listCol = ['antecedent support', 'consequent support', 'support', 'confidence', 'lift', 'leverage', 'conviction']
for i in listCol:
    saidaRecords[i] = saidaRecords[i].astype(str)
    saidaRecords[i] = saidaRecords[i].str.replace('.', ',')

## Ordena os dados baseado na confiança

In [503]:
# Ordena pela saidaRecords e salva em um novo DataFrame
saidaRecords = saidaRecords.sort_values(by='confidence', ascending=False)

## Salva os resultados em um arquivo .csv

In [504]:
# Salvando em um arquivo .csv
saidaRecords.to_csv(endereco + 'AprioriMlxtend_'+ 'ES_datatran' + anoArq + '.csv', encoding= 'ISO-8859-1', sep= ';', index=False)

In [505]:
# Gambiarra para atualizar o arquivo com os dados ordenados, de acordo com a célula anterior
# saidaRecords = pd.read_csv(endereco + 'AprioriMlxtend_'+ 'ES_datatran' + anoArq + '.csv', encoding='ISO-8859-1', sep=';')

In [506]:
# Salvando em um arquivo .csv
# saidaRecords.to_csv(endereco + 'AprioriMlxtend_'+ 'ES_datatran' + anoArq + '.csv', encoding= 'ISO-8859-1', sep= ';', index=False)

### Filtro para facilitar a visualização dos resultados

In [507]:
# rules[ (rules['antecedent_len'] >= 2) &
#        (rules['confidence'] > 0.75) &
#        (rules['lift'] > 1.2) ]

In [508]:
# rules[rules['antecedents'] == {'Eggs', 'Kidney Beans'}]

## Filtros modelos para serem utilizados no apriori

In [509]:
# saidaRecords[ (saidaRecords['antecedent_len'] >= 3 ) &
#                    (saidaRecords['consequent_len'] >= 3) ]

In [510]:
# frequent_itemsets[ (frequent_itemsets['length'] == 2) &
#                    (frequent_itemsets['support'] >= 0.8) ]

In [511]:
# frequent_itemsets[ frequent_itemsets['itemsets'] == {'Onion', 'Eggs'} ]

# ATIVIDADES FUTURAS

- Verificar se a coluna ['ignorados'] é necessária para analisar os resultados. <br>
Analisar o significado da coluna 'ignorados' no site da PRF. Ou no arquivo de descrição de tabelas.
***Resposta** : No link do anuário 2020, lá é apresentado alguns detalhes dos dados.

- Estudar os parâmetros do algoritmo apriori.

**[Resolvido]**
- Analisar a saída do 'lift' e 'conviction'. 
***Resposta**: O problema está no momento que os dados 'lift' e 'conviction' são passados para o arquivo .csv.

No Jupyter os dados estão sendo apresentados da maneira correta.

# Print Final

In [512]:
print("TUDO EXECUTADO!")

TUDO EXECUTADO!
